In [1]:
import requests
import scrapy
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
import string
import pandas as pd

url='https://internshala.com/fresher-jobs'
sel=Selector(text=BeautifulSoup(requests.get(url).content).prettify())
             
pages=sel.xpath('//span[@id="total_pages"]').xpath('normalize-space(./text())').extract()
pages[0]=int(pages[0])
print(pages[0])

4


In [2]:
class jobMan(scrapy.Spider):
    name='job'
    
    to_remove={0:["\n                ","\n               "],\
               1:['\n             ','\n            ']}
    
    def start_requests(self):
        urls="https://internshala.com/fresher-jobs/page-1"
        yield scrapy.Request(url=urls,callback=self.parse)
    
    def parse(self,response):

        def remove_(attr,remove):
            for j in remove:
                attr=[f.replace(j, "") for f in attr]
                
        ID=response.xpath('//div[@class="container-fluid individual_internship visibilityTrackerItem"]/@internshipid').extract()
        
        Job_Post = response.xpath('//div[@class="heading_4_5 profile"]/a').xpath('normalize-space(./text())').extract()
        
        Company = response.xpath('//a[@class="link_display_like_text"]').xpath('normalize-space(./text())').extract()

        ind_location=[]
        Location=[]
        for i in ID:
            ind_location.append(response.xpath("//div[@internshipid='" + i + "']//a[@class='location_link']").xpath('normalize-space(./text())').extract())
        for i in ind_location:
            s=''
            c=0
            if len(i)>1:
                for j in i:
                    if c>0:
                        s=s+', '+j
                    else:
                        s=s+j
                    c+=1
            else:
                s=j
            Location.append(s) 
            
        Apply_By = response.xpath('//div[@class="internship_other_details_container"]/div[@class="other_detail_item_row"][2]//div[@class="item_body"]').xpath('normalize-space(./text())').extract()

        for page in range(2,pages[0]+1):
            yield(scrapy.Request(url=f"https://internshala.com/fresher-jobs/page-{page}",callback=self.parse))
            
        yield {
            'ID': ID,
            'Job':Job_Post,
            'Company':Company,
            'Location':Location,
            'Apply_By':Apply_By
        }
        
        
process=CrawlerProcess(settings={
    'FEED_URI':'Unexpected.csv',
    'FEED_FORMAT':'csv'
})
process.crawl(jobMan)
process.start()

2021-07-09 22:38:21 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-07-09 22:38:21 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 21.2.0, Python 3.8.10 (default, May 19 2021, 13:12:57) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.19042-SP0
2021-07-09 22:38:21 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-07-09 22:38:21 [scrapy.crawler] INFO: Overridden settings:
{}
2021-07-09 22:38:21 [scrapy.extensions.telnet] INFO: Telnet Password: a68ac3a482216e32
2021-07-09 22:38:21 [py.warnings] WARNING: C:\Users\Eddie\anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2021-07

In [3]:
final=pd.read_csv('Unexpected.csv')
final

,ID,Job,Company,Location,Apply_By
0,NaN,"Marketing Executive,Embedded Systems Associate...","Lotrec AsiaPacific Private Limited,Sigsenz Tec...",NaN,"8 Aug' 21,8 Aug' 21,7 Aug' 21,7 Aug' 21,7 Aug'..."
1,NaN,"Flutter Developer,Business Development Special...","Techtonic Enterprises Private Limited,Claraeon...",NaN,"1 Aug' 21,31 Jul' 21,31 Jul' 21,30 Jul' 21,30 ..."
2,NaN,"Full Stack Developer,Associate Software Develo...","Nikulsan Technologies Private Limited,IQGatewa...",NaN,"14 Jul' 21,11 Jul' 21,11 Jul' 21,11 Jul' 21,11..."
3,NaN,Management Trainee - Assistant Category Manage...,"Noise,RoaDo,CrewKarma,Internshala,Mithi Softwa...",NaN,"24 Jul' 21,25 Jul' 21,24 Jul' 21,24 Jul' 21,23..."
